In [1]:
import time
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch, pykitti
import numpy as np
import sys
# sys.path.append("..")

from data_loader.kitti_odometry_dataset import KittiOdometryDataset
from model.monorec.monorec_model import MonoRecModel
from utils import unsqueezer, map_fn, to

target_image_size = (256, 512)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

dataset = KittiOdometryDataset("example/data/kitti", sequences=["07"], target_image_size=target_image_size, frame_count=2,
                               depth_folder="image_depth_annotated", lidar_depth=True, use_dso_poses=True,
                               use_index_mask=None)
dataset._dataset_sizes = [1000]
dataset._datasets[0].cam2_files = [f"example/data/kitti/sequences/07/image_2/{i:06d}.png" for i in range(dataset._dataset_sizes[0])]
dataset._datasets[0].cam3_files = [f"example/data/kitti/sequences/07/image_3/{i:06d}.png" for i in range(dataset._dataset_sizes[0])]
# print(dataset._datasets[0].cam3_files) # cam? corresponds to image_? folder

# batch, depth = dataset.__getitem__(164)


In [2]:
# len(batch['frames'])
ind=169
Img2=dataset._datasets[0].get_cam2(ind)
# Img2.show()
checkpoint_location = Path("saved/checkpoints/monorec_depth_ref.pth")

inv_depth_min_max = [0.33, 0.0025]

print("Initializing model...")
monorec_model = MonoRecModel(checkpoint_location=checkpoint_location, inv_depth_min_max=inv_depth_min_max)

monorec_model.to(device)
monorec_model.eval()

print("Fetching data...")
index = 164
# Corresponds to image index 169

batch, depth = dataset.__getitem__(index)
batch = map_fn(batch, unsqueezer)
depth = map_fn(depth, unsqueezer)

batch = to(batch, device)

print("Starting inference...")
s = time.time()
with torch.no_grad():
    data = monorec_model(batch)

c:\ProgramData\Anaconda3\envs\torch_17\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Initializing model...
Fetching data...
self._offset=5

Starting inference...
tensor([[-0.9923, -0.9884, -0.9845,  ...,  0.9983,  1.0023,  1.0062],
        [-0.9923, -0.9884, -0.9845,  ...,  0.9983,  1.0023,  1.0062],
        [-0.9923, -0.9884, -0.9845,  ...,  0.9983,  1.0023,  1.0062],
        ...,
        [-0.9910, -0.9871, -0.9832,  ...,  0.9959,  0.9999,  1.0038],
        [-0.9910, -0.9871, -0.9832,  ...,  0.9959,  0.9999,  1.0038],
        [-0.9910, -0.9871, -0.9832,  ...,  0.9959,  0.9998,  1.0038]],
       device='cuda:0')
tensor([[-1.0029, -0.9990, -0.9951,  ...,  0.9897,  0.9936,  0.9976],
        [-1.0030, -0.9991, -0.9951,  ...,  0.9897,  0.9936,  0.9976],
        [-1.0030, -0.9991, -0.9952,  ...,  0.9897,  0.9937,  0.9976],
        ...,
        [-1.0099, -1.0059, -1.0020,  ...,  0.9904,  0.9943,  0.9982],
        [-1.0099, -1.0060, -1.0020,  ...,  0.9904,  0.9943,  0.9982],
        [-1.0099, -1.0060, -1.0021,  ...,  0.9904,  0.9943,  0.9982]],
       device='cuda:0')


In [3]:
# a=torch.ones((10,3,4))
# b=a.expand(3,1,1)

c:\ProgramData\Anaconda3\envs\torch_17\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
